In [1]:
import tensorflow as tf
import numpy as np

from tensorflow import keras
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
# from tensorflow.keras.utils import np_utils
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras import layers 
import tensorflow.keras.backend as K 

from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler


import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
# import plotly.express as px
# import plotly.graph_objects as go

from math import sqrt

In [2]:
lstm_train_X = pd.read_csv("PCT\\PCT_train_x.csv")
lstm_train_y = pd.read_csv("PCT\\PCT_train_y.csv")

lstm_test_X = pd.read_csv("PCT\\PCT_test_x.csv")
lstm_test_y = pd.read_csv("PCT\\PCT_test_y.csv")

In [3]:
team = list(lstm_train_X.T_ID.unique())
year = list(lstm_train_y.YEAR.unique())

In [4]:
team, year

(['LG', 'HH', 'NC', 'HT', 'SK', 'KT', 'WO', 'LT', 'SS', 'OB'],
 [2016, 2017, 2018, 2019])

In [5]:
model_dict = dict()
hist_dict = dict()
test_pred_df = pd.DataFrame([],columns = ['YEAR','T_ID','y','y_pred','MSE','MSE_avg'])

idx = 0
for y in year:
    tmp1 = lstm_train_X[lstm_train_X["YEAR"] == y]
    tmp2 = lstm_train_y[lstm_train_y["YEAR"] == y]
    tmp3 = lstm_test_X[lstm_test_X["YEAR"] == y]
    tmp4 = lstm_test_y[lstm_test_y["YEAR"] == y]
    for t in team:
        name = '{}{}'.format(y,t)
        print(name,"=======================================")
        
        X_train = tmp1[tmp1["T_ID"] == t].drop(["T_ID","YEAR"],axis = 1)
        y_train = tmp2[tmp2["T_ID"] == t].drop(["T_ID","YEAR"],axis=1)
        X_test = tmp3[tmp3["T_ID"] == t].drop(["T_ID","YEAR"],axis=1)
        y_test = tmp4[tmp4["T_ID"] == t].drop(["T_ID","YEAR"],axis=1)
        
        X_train_v = X_train.values
        y_train_v = y_train.values

        X_test_v = X_test.values
        y_test_v = y_test.values
        """
        X_train_t = X_train_v.reshape(X_train_v.shape[0], 2,X_train_v.shape[1]//2)
        X_test_t = X_test_v.reshape(X_test_v.shape[0], 2,X_test_v.shape[1]//2)
        """
        
        ## model
        K.clear_session() 
        
        model = Sequential()
        optimizer = Adam(lr=0.01)
        model.add(Dense(1024, input_shape = [X_train.shape[1]]))
        model.add(Dense(1024))
        model.add(Dense(1024))
        model.add(Dense(1024))
        model.add(Dense(1024))
        model.add(Dense(1)) # output = 1
        model.compile(loss='mean_squared_error', optimizer=optimizer,metrics=['mae'])

       
        model.summary()
        
        early_stop = EarlyStopping(monitor='loss', mode = 'min',patience=2, verbose=1)

        hist1 = model.fit(X_train, y_train, epochs=100,
                  batch_size=1, verbose=1, callbacks=[early_stop])
        ##
        
        model_dict[name] = model
        hist_dict[name] = hist1
        
        y_pred = model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        
        ######### MSE AVG#########
        mse_avg = mean_squared_error(y_test,[y_train.mean()[0]])
        
        
        test_pred_df.loc[idx,:] = [y,t,y_test_v.reshape(-1)[0],y_pred.reshape(-1)[0], mse,mse_avg]

        idx += 1

test_pred_df[['y','y_pred',"shift_AVG_1","shift_AVG_2",'MSE','MSE_avg']] = test_pred_df[['y','y_pred',"shift_AVG_1","shift_AVG_2",'MSE','MSE_avg']].astype(float)

2016LG =======================================


W0927 20:31:43.176402 18632 training.py:504] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1024)              59392     
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_4 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 1025      
Total params: 4,258,817
Trainable params: 4,258,817
Non-trainable params: 0
______________________________________________

W0927 20:32:03.378823 18632 training.py:504] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>


Epoch 00012: early stopping
2016HH =======================================


...

W0927 20:44:15.352453 18632 training.py:504] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1024)              59392     
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_4 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 1025      
Total params: 4,258,817
Trainable params: 4,258,817
Non-trainable params: 0
______________________________________________

W0927 20:44:36.332367 18632 training.py:504] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>


Epoch 00015: early stopping


KeyError: "['shift_AVG_2', 'shift_AVG_1'] not in index"

In [6]:
test_pred_df

,YEAR,T_ID,y,y_pred,MSE,MSE_avg
0,2016,LG,0.608696,2.6844,4.30855,0.0198898
1,2016,HH,0.5,136.048,18373.2,0.00188918
2,2016,NC,0.565217,-11.6467,149.131,0.00236344
3,2016,HT,0.458333,-110.386,12286.5,0.00177981
4,2016,SK,0.458333,129.364,16616.6,1.59635e-05
5,2016,KT,0.291667,-933.407,871794,0.00425478
6,2016,WO,0.375,36.8734,1332.13,0.0412888
7,2016,LT,0.5,104.343,10783.3,0.0023538
8,2016,SS,0.5,-314.1,98973,0.00601803
9,2016,OB,0.666667,-391.222,153577,0.00438377
